In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_db_online.f_sale_prod_kdol_flat_non_dichoho
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
WITH
pos_sap_flat_data_original AS (
    SELECT
        b.calday,
        b.base_uom,
        b.sales_unit,
        b.transaction_index,
        b.transaction_id,
        b.product_id,
        b.business_unit,
        b.store_id,
        b.promotion,
        b.transaction_no,
        b.transaction_type,
        b.time_of_day,
        b.sale_time,
        b.credit_card_id,
        b.payment_method,
        b.department,
        b.pos_number,
        b.offer,
        b.discount_type,
        b.discount_type_group,
        b.recordmode,
        b.return_qty1,
        b.return_qty2,
        b.return_value,
        b.promotion_qty,
        b.base_sale_qty,
        b.receipt_count,
        b.sale_in_cost_value,
        b.promotion_sale,
        b.sale_qty,
        b.sale_amount,
        b.promotion_sale_l,
        b.return_sale_l,
        b.discount_item_no,
        b.discount_value,
        b.tax_amount,
        b.tender_value,
        b.vat_of_redeem,
         -- Bổ sung Kênh,
        IFNULL(c.channel,LTRIM('0',b.pos_number)) channel,
        2 AS priority
    FROM
        {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod_online_excl_dichoho b
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_ecart_channel c ON LTRIM('0',b.pos_number) = LTRIM('0',c.pos_code)
  UNION ALL
    SELECT
        a.calday,
        a.base_uom,
        a.sales_unit,
        a.transaction_index,
        a.transaction_id,
        a.product_id,
        a.business_unit,
        a.store_id,
        a.promotion,
        a.transaction_no,
        a.transaction_type,
        a.time_of_day,
        a.sale_time,
        a.credit_card_id,
        a.payment_method,
        a.department,
        a.pos_number,
        a.offer,
        a.discount_type,
        a.discount_type_group,
        a.recordmode,
        a.return_qty1,
        a.return_qty2,
        a.return_value,
        a.promotion_qty,
        a.base_sale_qty,
        a.receipt_count,
        a.sale_in_cost_value,
        a.promotion_sale,
        a.sale_qty,
        a.sale_amount,
        a.promotion_sale_l,
        a.return_sale_l,
        a.discount_item_no,
        a.discount_value,
        a.tax_amount,
        a.tender_value,
        0 AS vat_of_redeem,
        a.channel,
         -- Bổ sung Kênh,
        1 AS priority
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod_kdol_wmp a
    WHERE a.calday >= "2023-01-01"
    AND channel IN ('Winmart-Website')
),
pos_sap_flat_txn_removed_duplicated AS (
    SELECT
        * EXCEPT(rn)
    FROM (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY transaction_id ORDER BY priority ASC) rn
        FROM (SELECT DISTINCT transaction_id, priority FROM POS_SAP_FLAT_DATA_ORIGINAL)
    )
    WHERE rn = 1
)
SELECT
    a.*
FROM
    pos_sap_flat_data_original a
    INNER JOIN pos_sap_flat_txn_removed_duplicated b USING (transaction_id, priority)
""")